<a href="https://colab.research.google.com/github/Enrico-Call/RL-AKI/blob/create-notebook/Data_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/img/logo_amds.png?raw=1" alt="Logo" width=128px/>

# VUmc Research Project - Reinforcement Learning for Sepsis Prevention
# Data Extraction

AmsterdamUMCdb version 1.0.2 March 2020  
Copyright &copy; 2003-2022 Amsterdam UMC - Amsterdam Medical Data Science

## 1. Set up the environment variables for Colab and GoogleBigQuery to access

In [4]:
import os
from google.colab import auth
from google.cloud.bigquery import magics
from google.cloud import bigquery

#sets dateset
DATASET_PROJECT_ID = 'amsterdamumcdb-data'
DATASET_ID = 'ams102'
LOCATION = 'eu'

#sets the project id
PROJECT_ID = "rl-aki"

#all libraries check this environment variable, so set it:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

#sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config

#sets client options job configuration
client_options = {}
client_options['location'] = LOCATION
magics.context.bigquery_client_options = client_options

#BigQuery requires a separate config to prevent the 'BadRequest: 400 Cannot explicitly modify anonymous table' error message
job_config = bigquery.job.QueryJobConfig()

#sets default client settings by re-using the previously defined config
client = bigquery.Client(project=PROJECT_ID, location=LOCATION, default_query_job_config=def_config)

config_gbq = {'query': 
          {'defaultDataset': {
              "datasetId": DATASET_ID, 
              "projectId": DATASET_PROJECT_ID
              },
           'Location': LOCATION}
           }

auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
#get the amsterdamumcdb package from PiPy repository for use in Colab
!pip install amsterdamumcdb
import amsterdamumcdb as adb

     |████████████████████████████████| 199 kB 5.2 MB/s 


### 1.1. Available documentation for AmsterdamUMCdb tables.
Additional information including sample Python code in Jupyter Notebooks and [SQL](https://github.com/AmsterdamUMC/AmsterdamUMCdb/tree/master/amsterdamumcdb/sql) is available from the official [AmsterdamUMCdb GitHub repository](https://github.com/AmsterdamUMC/AmsterdamUMCdb) and [wiki](https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki). 

Table | Wiki | Example code
:- | :--| -:
admissions|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/admissions | [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/admissions.ipynb) 
drugitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/drugitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/drugitems.ipynb) 
freetextitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/freetextitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/freetextitems.ipynb)
listitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/lisitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/listitems.ipynb)
numericitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/numericitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/numericitems.ipynb)
procedureorderitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/procedureorderitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/procedureorderitems.ipynb)
processitems|https://github.com/AmsterdamUMC/AmsterdamUMCdb/wiki/processitems|[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmsterdamUMC/AmsterdamUMCdb/blob/master/tables/processitems.ipynb)

**Note**: In contrast to the publicly available version of AmsterdamUMCdb, we have moved *unvalidated* (device) data (`registeredby IS NULL`) from the `numericitems` table to a separate table `unvalidatednumericitems` to reduce query costs when unvalidated device data is not needed. However, depending on the chosen solution, you may need to combine the data from both tables.

## 2. Query AmsterdamUMCdb 

In [ ]:
data = pd.read_gbq(
    '''
    WITH sepsis AS (
      SELECT 
    l.admissionid
FROM listitems_validated l
LEFT JOIN admissions a ON
        l.admissionid = a.admissionid
 WHERE 
    (
        l.itemid = 15808  --Opname Sepsis
        AND (l.itemid = 16675  --DMC_Opname Sepsis
        )
        AND (l.value = 'Ja'))
GROUP BY l.admissionid
),
action AS (    SELECT
        d.admissionid
    FROM drugitems d
    LEFT JOIN admissions a ON
        d.admissionid = a.admissionid
    WHERE 
        d.ordercategoryid = 55 
        AND d.itemid IN (
            7229, -- Noradrenaline (Norepinefrine)
            7291  -- Crystalloid (fluid)
        )
        GROUP BY d.admissionid
    ),
  state as (
    SELECT n.admissionid,
    FROM numericitems_validated n
    LEFT JOIN admissions a ON
        n.admissionid = a.admissionid
    WHERE n.itemid IN (
    9941,  --Kreatinine (blood)
    8794,  --UrineCAD
    6642,  --ABP Mean Blood Pressure
    9924,  --Natrium in blood
    9927,  --Kalium in blood
    6836  --Kreatinine
    , 8796  --UrineSupraPubis
    , 8798 --UrineSpontaan
    , 8800 --UrineIncontinentie
    , 8803 --UrineUP
    , 10743 --Nefrodrain li Uit
    , 10745 --Nefrodrain re Uit
        ) 
      GROUP BY n.admissionid
  )
  SELECT 
    n.admissionid,
    n.itemid,
    n.item,
    n.value,
    n.registeredby,
    n.measuredat,
    (n.measuredat - a.admittedat) AS time,
    a.weightgroup,
    a.heightgroup,
    a.gender,
    a.agegroup,
    FROM numericitems_validated n
    LEFT JOIN admissions a ON
    n.admissionid = a.admissionid
    LEFT JOIN sepsis s ON
    n.admissionid = s.admissionid
    LEFT JOIN action v ON
    n.admissionid = v.admissionid
    LEFT JOIN state r ON 
    n.admissionid = r.admissionid
    WHERE n.itemid IN (
    9941,  --Kreatinine (blood)
    8794,  --UrineCAD
    6642,  --ABP Mean Blood Pressure
    9924,  --Natrium in blood
    9927,  --Kalium in blood
    6836  --Kreatinine
    , 8796  --UrineSupraPubis
    , 8798 --UrineSpontaan
    , 8800 --UrineIncontinentie
    , 8803 --UrineUP
    , 10743 --Nefrodrain li Uit
    , 10745 --Nefrodrain re Uit
    ) 
    '''
    , configuration=config_gbq)